In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from osgeo import gdal,ogr,osr

In [1]:
import numpy as np
from osgeo import gdal, gdal_array
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Step 1: Load your image using GDAL
img_file = "input/ang20180812t232708rfl/ang20180812t232708_rfl_v2r2_img"
hdr_file = "input/ang20180812t232708rfl/ang20180812t232708_rfl_v2r2_img.hdr"
img_open = gdal.Open(img_file)

# Step 2: Read the number of bands, width, and height
nbands = img_open.RasterCount  # Number of bands (425 in your case)
ncols = img_open.RasterXSize   # Width of the image (9426 in your case)
nrows = img_open.RasterYSize   # Height of the image (659 in your case)

# Step 3: Read all bands into a 3D NumPy array
# Initialize an empty NumPy array of shape (nbands, nrows, ncols)
img_array = np.zeros((nbands, nrows, ncols), dtype=np.float32)

# Loop through each band and read it into the array
for i in range(nbands):
    band = img_open.GetRasterBand(i + 1)  # Band indices in GDAL start from 1
    img_array[i, :, :] = band.ReadAsArray()

# Now, img_array has the shape (425, 659, 9426)

# alternatively
# Read raster data as numeric array from file
#rasterArray = gdal_array.LoadFile(img_file)


In [14]:
# Step 4: Reshape the array from (nbands, nrows, ncols) to (nrows * ncols, nbands)
reshaped_img = img_array.reshape((nbands, nrows * ncols)).T  # Shape: (659 * 9426, 425)

In [15]:
# Step 5: Standardize the data
scaler = StandardScaler()
standardized_img = scaler.fit_transform(reshaped_img)

In [18]:
# Step 6: Apply PCA
pca = PCA(n_components=0.95)  # Keep enough components to explain 95% variance
pca_img = pca.fit_transform(standardized_img)

# Step 7: Reshape back to the image format with reduced components
n_components = pca_img.shape[1]  # Number of principal components
reduced_img = pca_img.reshape((nrows, ncols, n_components))  # Shape: (659, 9426, n_components)

# Step 8: Visualize the first 3 principal components as an RGB image
plt.imshow(reduced_img[:, :, :3])  # Display the first 3 components as RGB
plt.show()

In [ ]:
import numpy as np
from osgeo import gdal
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Step 1: Open the hyperspectral image
img_file = "input/ang20180812t232708rfl/ang20180812t232708_rfl_v2r2_img"
img_open = gdal.Open(img_file)

# Read image bands into a NumPy array (nbands, nrows, ncols)
nbands = img_open.RasterCount
ncols = img_open.RasterXSize
nrows = img_open.RasterYSize

# Create a NumPy array and read all bands into it
img_array = np.zeros((nbands, nrows, ncols), dtype=np.float32)
for i in range(nbands):
    band = img_open.GetRasterBand(i + 1)
    img_array[i, :, :] = band.ReadAsArray()

# Step 2: Reshape to (nrows * ncols, nbands)
reshaped_img = img_array.reshape((nbands, nrows * ncols)).T

# Step 3: Replace -9999 with NaN
reshaped_img[reshaped_img == -9999] = np.nan

# Step 4: Impute NaN values with the mean of the bands (or other strategies)
imputer = SimpleImputer(strategy='mean')  # You could use 'median' or 'most_frequent'
imputed_img = imputer.fit_transform(reshaped_img)

# Step 5: Standardize the data (important for PCA to work properly)
scaler = StandardScaler()
standardized_img = scaler.fit_transform(imputed_img)

# Step 6: Perform PCA
pca = PCA(n_components=0.95)  # Keep enough components to explain 95% of the variance
pca_img = pca.fit_transform(standardized_img)

# Step 7: Reshape back to the image format for visualization
n_components = pca_img.shape[1]
reduced_img = pca_img.reshape((nrows, ncols, n_components))

# Visualize the first 3 components as an RGB image (optional)
plt.imshow(reduced_img[:, :, :3])
plt.title('First 3 Principal Components as RGB')
plt.show()

# You can further analyze or visualize the PCA results
